<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# Neural Collaborative Filtering

Neural Collaborative Filtering (NCF) is a well known recommendation algorithm that generalizes the matrix factorization problem with multi-layer perceptron. 

This notebook provides an example of how to utilize and evaluate NCF implementation in the `recommenders`. We use a smaller dataset in this example to run NCF efficiently with GPU acceleration.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pandas as pd
import tensorflow as tf
# tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (
    map, ndcg_at_k, precision_at_k, recall_at_k
)
from recommenders.utils.notebook_utils import store_metadata

from datasets import outfits

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.9.23 | packaged by conda-forge | (main, Jun  4 2025, 18:02:02) 
[Clang 18.1.8 ]
Pandas version: 2.3.3
Tensorflow version: 2.14.0


/opt/homebrew/Caskroom/miniforge/base/envs/ai_ml-ncf/lib/python3.9/site-packages/pandera/_pandas_deprecated.py:160: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


Set the default parameters.

In [3]:
# top k items to recommend
TOP_K = 10

# Change data size as appropriate
OUTFITS_DATA_SIZE = '100'

# Model parameters
EPOCHS = 100
BATCH_SIZE = 256

SEED = 42

### 1. Download the dataset

In [4]:
df = outfits.load_pandas_df(
    header=["UserId", "Weather", "Clothing", "Rating"],
    filepath=f"datasets/csv/example_feature1.csv"
)
df['ClothingId'] = df['Clothing'].astype('category').cat.codes
print(df)

    UserId Weather Clothing  Rating  ClothingId
0        1   Humid   Blazer     2.2           0
1        1   Rainy   Blazer     2.8           0
2        1   Sunny   Hoodie     2.5           5
3        1   Sunny    Jeans     3.8           6
4        2  Cloudy   Hoodie     4.1           5
..     ...     ...      ...     ...         ...
95      19   Windy   Shorts     2.5          10
96      20   Humid     Polo     3.7           9
97      20   Snowy   Hoodie     5.0           5
98      20   Sunny   Chinos     3.3           2
99      20   Sunny  Joggers     2.4           7

[100 rows x 5 columns]


### 2. Split the data using the Spark splitter provided in utilities

In [5]:
train, test = python_random_split(
    df, 
    ratio=0.75
)
train = train[train['Rating'] > 0]
assert len(train) > 0, "STOP: Training set is empty immediately after splitting."

Filter out any users or items in the test set that do not appear in the training set.

In [6]:
test = test[test["UserId"].isin(train["UserId"].unique())]
test = test[test["ClothingId"].isin(train["ClothingId"].unique())]

train_sorted = train.sort_values(by="UserId")
test_sorted = test.sort_values(by="UserId")

Write datasets to csv files.

In [7]:
train_file = "./train.csv"
test_file = "./test.csv"
train_sorted.to_csv(train_file, index=False)
test_sorted.to_csv(test_file, index=False)

Generate an NCF dataset object from the data subsets.

In [8]:
data = NCFDataset(
  train_file=train_file, 
  test_file=test_file, 
  seed=SEED, 
  col_user="UserId", 
  col_item="ClothingId", 
  col_rating="Rating")

assert data.n_users > 0 and data.n_items > 0, "STOP: The NCFDataset object loaded no users or items."

INFO:recommenders.models.ncf.dataset:Indexing ./train.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./test.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./test_full.csv ...


### 3. Train the NCF model on the training data, and get the top-k recommendations for our testing data

NCF accepts implicit feedback and generates prospensity of items to be recommended to users in the scale of 0 to 1. A recommended item list can then be generated based on the scores. Note that this quickstart notebook is using a smaller number of epochs to reduce time for training. As a consequence, the model performance will be slighlty deteriorated. 

In [9]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

/opt/homebrew/Caskroom/miniforge/base/envs/ai_ml-ncf/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2025-10-12 06:37:28.212414: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled


In [10]:
# Available for use with larger datasets
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time))

Took 0.5507 seconds for training.


In [11]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.ClothingId.unique())
    for user in train.UserId.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"UserId": users, "ClothingId":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["UserId", "ClothingId"], how="outer")
    all_predictions = merged[merged.Rating.isnull()].drop('Rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 0.0202 seconds for prediction.


### 4. Evaluate how well NCF performs

The ranking metrics are used for evaluation.

In [12]:
eval_map = map(test, all_predictions, col_prediction='prediction', k=TOP_K, col_user='UserId', col_item='ClothingId')
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K, col_user='UserId', col_item='ClothingId', col_rating='Rating')
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K, col_user='UserId', col_item='ClothingId')
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K, col_user='UserId', col_item='ClothingId')

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.323413
NDCG:	0.423614
Precision@K:	0.113333
Recall@K:	0.666667


In [13]:
# Record results for tests - ignore this cell
store_metadata("map", eval_map)
store_metadata("ndcg", eval_ndcg)
store_metadata("precision", eval_precision)
store_metadata("recall", eval_recall)
store_metadata("train_time", train_time.interval)
store_metadata("test_time", test_time.interval)